In [1]:
import os
import torch
from torch.utils.data import DataLoader
from Configs import Config
from model import  Imporved_BRIO
import utils
from TestDataSet import TestDataSet
from compare_mt.rouge.rouge_scorer import RougeScorer
rouge_scorer = RougeScorer(['rouge1', 'rouge2', 'rougeLsum'], use_stemmer=True)

DATA_NAME = 'xsum'
args = Config(DATA_NAME)
pair_data_path = f'./evaluate/E1-{DATA_NAME}-res-datas'
if(not os.path.exists(pair_data_path)):
    os.mkdir(pair_data_path)
    
#MODEL_PATH = f'./check_points/xxxxxxxxxxx.bin'

hyps_txt = os.path.join(pair_data_path,'xsum-brio-hyps.txt')
refs_txt = os.path.join(pair_data_path,'xsum-brio-ref.txt')
def flush_hyps(x):
    with open(hyps_txt,'a+',encoding='utf-8') as f:
        f.write(x + '\n')
        f.close()
def flush_refs(x):
    with open(refs_txt,'a+',encoding='utf-8') as f:
        f.write(x + '\n')
        f.close()
hyps_txt,refs_txt

('./evaluate/E1-xsum-res-datas/xsum-brio-hyps.txt',
 './evaluate/E1-xsum-res-datas/xsum-brio-ref.txt')

In [2]:
model =torch.load(MODEL_PATH).to(args.device)
model.eval()
tokenizer = model.tokenizer


In [3]:
model.model.model.generation_mode()

In [4]:
validation_set = TestDataSet('./xsum_test_set/')
validation_loader =DataLoader(dataset=validation_set,batch_size=50,shuffle=False,drop_last=False)

In [ ]:

for k,data in enumerate(validation_loader):
    print(f'[{k}/{len(validation_loader)}]')
    if(DATA_NAME == 'xsum'):
        x = [ i.strip() for i in data[0]]
        y = [ i.strip() for i in data[1]]
    else :
        x = [ i.strip().lower() for i in data[0]]
        y = [ i.strip().lower() for i in data[1]]
    pred_list=[]
    X = tokenizer.batch_encode_plus(x,max_length=args.total_len, truncation=True, return_tensors='pt',padding='max_length').to(args.device)
    with torch.no_grad():
        out = model.model.generate(
                input_ids=X.input_ids.to(args.device),
                attention_mask=X.attention_mask.to(args.device),
                max_length = args.gen_max_len + 2,
                min_length=args.gen_min_len + 1,
                no_repeat_ngram_size=3,
                early_stopping=True,
                num_beams=4,
                length_penalty= args.length_penalty
            )
    pred = tokenizer.batch_decode(out.cpu(),skip_special_tokens=True,clean_up_tokenization_spaces=True)
    if  DATA_NAME == 'xsum':
        for x in pred:
            flush_hyps(x.strip())
    else:
        for x in pred:
            flush_hyps(x.strip().lower())
    for l in y:
        flush_refs(l)